In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/msoler/Repos/mlops-zoomcamp-fork/notebooks/mlruns/1', creation_time=1684777764384, experiment_id='1', last_update_time=1684777764384, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
file_jan = "data/green_tripdata_2021-01.parquet"
file_feb = "data/green_tripdata_2021-02.parquet"

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    return df

In [7]:
df_train = read_dataframe(file_jan)
df_val = read_dataframe(file_feb)

In [8]:
df_train["PU_DO"] = df_train["PULocationID"] + '_' + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + '_' + df_val["DOLocationID"]

In [9]:
categorical = ['PU_DO'] #['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

## Experiment tracking

In [11]:
with mlflow.start_run():
    
    mlflow.set_tag("first tag", "first run")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("test-data-path", "data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
        
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

In [13]:
with mlflow.start_run():
    
    mlflow.set_tag("second run", "Linear Regression")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("test-data-path", "data/green_tripdata_2021-02.parquet")
        
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
        
    mlflow.log_artifact("models/lin_reg.bin", artifact_path="models_pickle")

In [14]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[20:36:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.25845                          
[1]	validation-rmse:7.99452                           
[2]	validation-rmse:7.05246                           
[3]	validation-rmse:6.76330                           
[4]	validation-rmse:6.66372                           
[5]	validation-rmse:6.62612                           
[6]	validation-rmse:6.60293                           
[7]	validation-rmse:6.59174                           
[8]	validation-rmse:6.58545                           
[9]	validation-rmse:6.58046                           
[10]	validation-rmse:6.57229                          
[11]	validation-rmse:6.56876                          
[12]	validation-rmse:6.56656                          
[13]	validation-rmse:6.56163                          
[14]	validation-

## Model management

https://mlflow.org/docs/latest/tracking.html#automatic-logging

In [36]:
params = {
    "learning_rate": 0.23829777737930935,
    "max_depth": 10,
    "min_child_weight": 2.895275554615752,
    "objective": 'reg:linear',
    "reg_alpha": 0.013860303317771881,
    "reg_lambda": 0.02621444004427755,
    "seed": 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2023/05/30 20:25:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '92441e30d0d24c7f9b19088d22a93bfc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[20:25:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.99607
[1]	validation-rmse:13.91698
[2]	validation-rmse:11.68787
[3]	validation-rmse:10.10565
[4]	validation-rmse:9.00337
[5]	validation-rmse:8.25025
[6]	validation-rmse:7.74152
[7]	validation-rmse:7.39641
[8]	validation-rmse:7.16565
[9]	validation-rmse:7.00575
[10]	validation-rmse:6.89626
[11]	validation-rmse:6.81899
[12]	validation-rmse:6.76266
[13]	validation-rmse:6.72169
[14]	validation-rmse:6.69412
[15]	validation-rmse:6.67067
[16]	validation-rmse:6.65644
[17]	validation-rmse:6.64380
[18]	validation-rmse:6.63502
[19]	validation-rmse:6.62835
[20]	validation-rmse:6.62271
[21]	validation-rmse:6.61779
[22]	validation-rmse:6.61255
[23]	validation-rmse:6.61022
[24]	validation-rmse:6.60722
[25]	validation-rmse:6.60269
[26]	validation-rmse:6.60057
[27]	vali

2023/05/30 20:25:34 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [26]:
mlflow.xgboost.autolog(disable=True)

In [35]:
with mlflow.start_run():
    best_params = {
        "learning_rate": 0.23829777737930935,
        "max_depth": 10,
        "min_child_weight": 2.895275554615752,
        "objective": 'reg:linear',
        "reg_alpha": 0.013860303317771881,
        "reg_lambda": 0.02621444004427755,
        "seed": 42
    }

    mlflow.log_params(best_params)

    mlflow.xgboost.autolog()

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[20:24:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.99607
[1]	validation-rmse:13.91698
[2]	validation-rmse:11.68787
[3]	validation-rmse:10.10565
[4]	validation-rmse:9.00337
[5]	validation-rmse:8.25025
[6]	validation-rmse:7.74152
[7]	validation-rmse:7.39641
[8]	validation-rmse:7.16565
[9]	validation-rmse:7.00575
[10]	validation-rmse:6.89626
[11]	validation-rmse:6.81899
[12]	validation-rmse:6.76266
[13]	validation-rmse:6.72169
[14]	validation-rmse:6.69412
[15]	validation-rmse:6.67067
[16]	validation-rmse:6.65644
[17]	validation-rmse:6.64380
[18]	validation-rmse:6.63502
[19]	validation-rmse:6.62835
[20]	validation-rmse:6.62271
[21]	validation-rmse:6.61779
[22]	validation-rmse:6.61255
[23]	validation-rmse:6.61022
[24]	validation-rmse:6.60722
[25]	validation-rmse:6.60269
[26]	validation-rmse:6.60057
[27]	vali

2023/05/30 20:24:27 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [38]:
logged_model = 'runs:/59a9b3cb47d04f689457a47ae50756be/models_mlflow'

loaded_model = mlflow.pyfunc.load_model(logged_model)

xgboost_model = mlflow.xgboost.load_model(logged_model)

2023/05/30 20:27:01 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[20:27:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[20:27:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [39]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 59a9b3cb47d04f689457a47ae50756be

In [40]:
xgboost_model

In [33]:
y_pred = xgboost_model.predict(valid)

In [34]:
y_pred

array([14.729096 ,  7.2546597, 13.302215 , ..., 13.678202 ,  6.9842377,
        8.285766 ], dtype=float32)

## Model registry

In [41]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

Creating an experiment

In [46]:
client.create_experiment(name='test')

'2'

Searching runs

In [51]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 6.4',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [52]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: aeeec2b1c99d4d2e9172d9d7ffe0d2c5, rmse: 6.3068
run id: a6da789e59754b75a1f246c608c4be81, rmse: 6.3152
run id: f3c5f78bade24555af0e7ac72b59e1f1, rmse: 6.3155
run id: ce3ee4b614b743538468d17fd5778e37, rmse: 6.3220
run id: 5ec139b9b8f1472f98112e28e2e96873, rmse: 6.3277


Registering models

In [53]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [54]:
run_id = "aeeec2b1c99d4d2e9172d9d7ffe0d2c5"
model_uri = f"runs/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/30 20:41:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685472109239, current_stage='None', description=None, last_updated_timestamp=1685472109239, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/aeeec2b1c99d4d2e9172d9d7ffe0d2c5/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

Transitioning models between stages

In [56]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [58]:
model_version = 3
stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685472109239, current_stage='Staging', description=None, last_updated_timestamp=1685472508799, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/aeeec2b1c99d4d2e9172d9d7ffe0d2c5/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [60]:
from datetime import datetime

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {stage} on {datetime.today().date()}"
)

<ModelVersion: aliases=[], creation_timestamp=1685472109239, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-05-30', last_updated_timestamp=1685472569606, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/aeeec2b1c99d4d2e9172d9d7ffe0d2c5/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [ ]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

/var/folders/42/f9s_rgk15078ym2w50_xtc180000gq/T/ipykernel_5486/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [ ]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/cristian.martinez/Repositories/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [ ]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


{'rmse': 6.659623830022514}

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>